# Kaggle Bosch Production Line Challenge Model
## Summary
This is a [TensorFlow](https://www.tensorflow.org) model, built on a [Spark](https://spark.apache.org) framework, to attempt to solve the [Bosch Production Line Performance Challenge](https://www.kaggle.com/c/bosch-production-line-performance) on [Kaggle](https://www.kaggle.com).  This project was begun by Thomas Hughes on November 24, 2016, after the competition was completed.  It should be considered a test of effectiveness of technology platforms.

## Set Up Spark

In [1]:
# TODO: Setup Spark

## Import Bosch Data

In [2]:
# TODO: Import Bosch Data

## Data Wrangling

### Scale Data

In [3]:
# TODO: Scale Data

### Normalize Data

In [4]:
# TODO: Normalize Data on Logarithmic Scale

### Drop Outliers

In [5]:
# TODO: Drop Outliers in Training Set

### Reduce Dimensionality

#### Feature Selection

In [6]:
# TODO: Feature Selection; use Spark libraries if possible

#### Independent Component Analysis (?)

In [7]:
# TODO: Engineer some independent components using ICA

#### Principal Component Analysis

In [8]:
# TODO: Engineer some principal components using PCA

## TensorFlow

In [9]:
# TODO: TensorFlow Model

## Scale: Spark Context to EC2

In [10]:
# TODO: Switch Spark Context to EC2

## Kaggle Submission

In [11]:
# TODO: Submit predictions to Kaggle